# Regression Assignment

* You are required to use the below dataset on house prices to train a regression model:

https://www.kaggle.com/datasets/vikrishnan/boston-house-prices

#### Requirements:

- Divide dataset after importing into Train/Cross Validation/Test

- Test various hypothesis with different degrees and optimize your model and parameters using cross validation & test sets as explained in the lecture

- Apply Regularization and optimize the value of its parameter

##### Importing libraries

In [2]:
import pandas as pd
import numpy as np

##### Importing and preparing dataset

In [5]:
data = pd.csv_read('housing.csv')

AttributeError: module 'pandas' has no attribute 'csv_read'